# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
def priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status):
    """(str, int, int, float, int, init, , float, float, str, str, str)->float
    Using the provided algorithm, this function computes the PRIEST
    COVID Clinical Severity Score.  This score is transplated using the provided
    table to compute a 30-day probability of adverse outcomes, which
    the function returns.
    
    See https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence for 
    further documentation.
    """
    
    if sex.lower() == 'female':
        pts_sex = 0
    elif sex.lower() == 'male':
        pts_sex = 1
    else:
        pts_sex = None
        raise ValueError(f'Invalid value for sex: "{sex}". Sex must be male or female.')
        
    if age < 16:
        pts_age = None
    elif age <= 49:
        pts_age = 0
    elif age <= 65:
        pts_age = 2
    elif age <= 80:
        pts_age = 3
    else:
        pts_age = 4
        
    if breath <9:
        pts_breath = 3
    elif breath <= 11:
        pts_breath = 1
    elif breath <= 20:
        pts_breath = 0
    elif breath <= 24:
        pts_breath = 2
    else:
        pts_breath = 3
        
    if o2sat > 0.95:
        pts_o2 = 0
    elif o2sat >= 0.94:
        pts_o2 = 1
    elif o2sat >= 0.92:
        pts_o2 = 2
    else:
        pts_o2 = 3
    
    if heart < 41:
        pts_heart = 3
    elif heart <= 50:
        pts_heart = 1
    elif heart <= 90:
        pts_heart = 0
    elif heart <= 110:
        pts_heart = 1
    elif heart <= 130:
        pts_heart = 2
    else:
        pts_heart = 3
        
    if systolic < 91:
        pts_systolic = 3
    elif systolic <= 100:
        pts_systolic = 2
    elif systolic <= 110:
        pts_systolic = 1
    elif systolic <= 219:
        pts_systolic = 0
    else:
        pts_systolic = 3
        
    if temp < 35.1:
        pts_temp = 3
    elif temp <= 36.0:
        pts_temp = 1
    elif temp <= 38:
        pts_temp = 0
    elif temp <= 39:
        pts_temp = 1
    else:
        pts_temp = 2
        
    if alertness.lower() == 'alert':
        pts_alertness = 0
    elif alertness.lower() == 'confused or not alert':
        pts_alertness = 3
    else:
        pts_alertness = None
        raise ValueError(f'Invalidi value for alertness: "{alertness}". Must be "alert" or "confused or not alert".')
        
    if inspired.lower() == 'air':
        pts_inspired = 0
    elif inspired.lower() == 'supplemental oxygen':
        pts_inspired = 2
    else:
        pts_inspired = None
        raise ValueError(f'Invalid value for inspired: "{inspired}". Must be "air" or "supplemental oxygen".')
        
    if status.lower() == 'unrestricted normal activity':
        pts_status = 0
    elif status.lower() == 'limited strenuous activity, can do light activity':
        pts_status = 1
    elif status.lower() == 'limited activity, can self-care':
        pts_status = 2
    elif status.lower() == 'limited self-care':
        pts_status = 3
    elif status.lower() == 'bed/chair bound, no self-care':
        pts_status = 4
    else:
        pts_status = None
        raise ValueError(f'Invalid value for status: "status". See documentation for valid values.')
    
#     score = pts_sex + pts_age + pts_breath + pts_o2 + pts_heart + pts_systolic + pts_temp + pts_alertness + pts_inspired + pts_status
    try:
        score = pts_sex + pts_age + pts_breath + pts_o2 + pts_heart + pts_systolic + pts_temp + pts_alertness + pts_inspired + pts_status
#         print(f'Score was {score}')
    except:
        score = None
        raise ValueError(f'Unable to compute score from given inputs. Partial scores were {pts_sex}, {pts_age}, {pts_breath}, {pts_02}, {pts_heart}, {pts_systolic}, {pts_temp}, {pts_alertness}, {pts_inspired}, {pts_status}')
        
    score_map = {
        0: 0.01,
        1: 0.01,
        2: 0.02,
        3: 0.02,
        4: 0.03,
        5: 0.09,
        6: 0.15,
        7: 0.18,
        8: 0.22,
        9: 0.26,
        10: 0.29,
        11: 0.34,
        12: 0.38,
        13: 0.46,
        14: 0.47,
        15: 0.49,
        16: 0.55
    }
    
        
    if score >= 26:
        risk = 0.99
    elif score >= 17:
        risk = 0.59
    else:
        risk = score_map[score]
        
    return (
        [pts_sex, pts_age, pts_breath, pts_o2, pts_heart, pts_systolic, pts_temp, pts_alertness, pts_inspired, pts_status],
        score, risk
    )
    
        
        


In [2]:
priest('male',34,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')

([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, 0.01)

In [3]:
priest('male',68,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')

([1, 3, 0, 0, 0, 0, 0, 0, 0, 0], 4, 0.03)

In [4]:
priest('gunk',34,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')

ValueError: Invalid value for sex: "gunk". Sex must be male or female.

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [5]:
import requests
import json

def find_hospital(age, sex, risk):
    """(int, str, float)->str
    Use the professor's REST web service to lookup and return the
    name of the hospital where this person should be sent.
    
    >>> find_hospital(40, 'male', 0.1)
    Southwest Hospital and Medical Center
    """
    
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    
    try:
        r = requests.get(url)
        info = r.json()
        hospital = info.get('hospital')
    except:
        hospital = None
        raise ValueError(f'Unable to find hospital for inputs: {age} {sex} {risk}')
    
    return hospital

In [6]:
find_hospital(40, 'male', 0.1)

'Southwest Hospital and Medical Center'

In [7]:
find_hospital()

TypeError: find_hospital() missing 3 required positional arguments: 'age', 'sex', and 'risk'

## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [8]:
import requests
from bs4 import BeautifulSoup

def get_address(hospital):
    """(str)->str
    Fetches the hospital address from the known webpage.
    
    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'
    """
    
    if hospital is None:
        raise ValueError(f'Cannot search for None as hospital')
    
    r = requests.get('https://www.officialusa.com/stateguides/health/hospitals/georgia.html')
    soup = BeautifulSoup(r.text)
    
    hospitals = {}
    for row in soup.find_all('tr'):
        data = list(row.find_all('td'))
        if len(data) > 0:
            (city, name, typ, address, website, beds, naixs_desc) = [d.text for d in data]
            if city.upper() == 'ATLANTA' and name.upper() == hospital.upper():
                hospitals[hospital] = address
                
    if len(hospitals) > 1:
        raise KeyError(f'Multiple addresses found for {hospital}')
    elif len(hospitals) < 1:
        raise KeyError(f'Hospital not found: {hospital}')

    result = hospitals.get(list(hospitals.keys())[0])
    return result
    

In [9]:
get_address('Southwest Hospital and Medical Center')

'501 FAIRBURN ROAD SW'

In [10]:
get_address('BJC Hospital')

KeyError: 'Hospital not found: BJC Hospital'

## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.



In [11]:
import csv

def process_people(file):
    result = {}

    with open(file) as f:
        reader = csv.reader(f, delimiter='|')
        for row in reader:
            (patient, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status) = row

            if (patient != 'patient'):
                age = int(age)
                breath = float(breath)
                o2sat = float(o2sat)
                heart = float(heart)
                systolic = float(systolic)
                temp = float(temp)
                scores, score, percentile = priest(sex, int(age), float(breath), float(o2sat), float(heart), float(systolic), float(temp), alertness, inspired, status)
                hospital = find_hospital(int(age), sex, float(percentile))
                if hospital is None:
                    print(f'Unable to find hospital for {sex}, {age}, {score}, {percentile}')
                location = get_address(hospital)
#                 result[patient] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, scores, score, percentile, hospital, location]
                result[patient] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, percentile, hospital, location]
    
    
    
    return result

In [ ]:
outputs = process_people('people.psv')

In [ ]:
import json

with open('people_results.json', 'w') as f:
    f.write(json.dumps(outputs, indent=4))


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [1]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main d88bc5b] Submitting the week 7 programming exercises
 2 files changed, 297 insertions(+), 8 deletions(-)
 create mode 100644 week07/week07_assignment_2.ipynb
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 3.66 KiB | 3.66 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:paulboal/hds5210-2022.git
   fc2200c..d88bc5b  main -> main
